# Datasets unification

Unification of all datasets into one.

Loading env variables:

In [ ]:
import os
import sys
from dotenv import load_dotenv

load_dotenv(override=True)
print('ENV variables loaded successfully!')

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Loading datasets files:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lib.env_var_keys import EnvVarKeys

# Get the datasets files paths
crianca_file_path = os.getenv(EnvVarKeys.CRIANCA_DATASET_ORIGINAL_FILE_PATH_KEY.value)
achado_anormal_file_path = os.getenv(EnvVarKeys.ACHADO_ANORMAL_DATASET_ORIGINAL_FILE_PATH_KEY.value)
tipo_morbidade_file_path = os.getenv(EnvVarKeys.TIPO_MORBIDADE_DATASET_ORIGINAL_FILE_PATH_KEY.value)
parto_file_path = os.getenv(EnvVarKeys.PARTO_DATASET_ORIGINAL_FILE_PATH_KEY.value)
gestacao_file_path = os.getenv(EnvVarKeys.GESTACAO_DATASET_ORIGINAL_FILE_PATH_KEY.value)
mulher_file_path = os.getenv(EnvVarKeys.MULHER_DATASET_ORIGINAL_FILE_PATH_KEY.value)
dado_evolutivo_file_path = os.getenv(EnvVarKeys.DADO_EVOLUTIVO_DATASET_ORIGINAL_FILE_PATH_KEY.value)


print(f"""Loading original datasets from: 
  -> crianca_file_path={crianca_file_path}
  -> achado_anormal_file_path={achado_anormal_file_path}
  -> tipo_morbidade_file_path={tipo_morbidade_file_path}
  -> parto_file_path={parto_file_path}
  -> gestacao_file_path={gestacao_file_path}
  -> mulher_file_path={mulher_file_path}
  -> dado_evolutivo_file_path={dado_evolutivo_file_path}""")

# Loading CSV files into dataframes
df_crianca = pd.read_csv(crianca_file_path, sep=',')
df_achado_anormal = pd.read_csv(achado_anormal_file_path, sep=',')
df_tipo_morbidade = pd.read_csv(tipo_morbidade_file_path, sep=',')
df_parto = pd.read_csv(parto_file_path, sep=',')
df_gestacao = pd.read_csv(gestacao_file_path, sep=',')
df_mulher = pd.read_csv(mulher_file_path, sep=',')
df_dado_evolutivo = pd.read_csv(dado_evolutivo_file_path, sep=',')


print(f"""Original datasets shapes:
  -> df_crianca.shape={df_crianca.shape}
  -> df_achado_anormal.shape={df_achado_anormal.shape}
  -> df_tipo_morbidade.shape={df_tipo_morbidade.shape}
  -> df_parto.shape={df_parto.shape}
  -> df_gestacao.shape={df_gestacao.shape}
  -> df_mulher.shape={df_mulher.shape}
  -> df_dado_evolutivo.shape={df_dado_evolutivo.shape}""")

## Datasets unification

Removing duplicated rows:

In [ ]:
from lib.dataframe_helper import remove_duplicated_rows

df_crianca = remove_duplicated_rows(df_crianca, 'df_crianca')
df_achado_anormal = remove_duplicated_rows(df_achado_anormal, 'df_achado_anormal')
df_tipo_morbidade = remove_duplicated_rows(df_tipo_morbidade, 'df_tipo_morbidade')
df_parto = remove_duplicated_rows(df_parto, 'df_parto')
df_gestacao = remove_duplicated_rows(df_gestacao, 'df_gestacao')
df_mulher = remove_duplicated_rows(df_mulher, 'df_mulher')
df_dado_evolutivo = remove_duplicated_rows(df_dado_evolutivo, 'df_dado_evolutivo')

Counting of `mc_cri_vdrl` column from `df_crianca_v1` dataframe:

In [ ]:
from lib.dataframe_helper import vdrl_count

vdrl_count(df_crianca, 'df_crianca')

Removendo as linhas desativadas de `df_crianca` (`df_crianca_v1['ativo'] != False`):

In [ ]:
vdrl_count(df_crianca)

deactivate_count = df_crianca[df_crianca['ativo'] == False].shape[0]
print(f'Removing {deactivate_count} deactivated rows.')

df_crianca = df_crianca[df_crianca['ativo'] != False] 
vdrl_count(df_crianca)

Converting negative VDRL test result (`mc_cri_vdrl = 1`) and empty VDRL test result to positive VDRL test result (`mc_cri_vdrl = 0`)  if there is a link to the abnormal finding **428** (`O98.1 Sifilis complic gravidez parto e puerperio`):

In [ ]:
achado_anormal_id = 428

df_achado_anormal = df_achado_anormal[df_achado_anormal['mc_aac_id'] == achado_anormal_id]

df_crianca.loc[df_crianca['mc_pes_id'].isin(df_achado_anormal['mc_cri_id'].to_list()), 'mc_cri_vdrl'] = 0

vdrl_count(df_crianca, 'df_crianca')

Converting negative VDRL test result (`mc_cri_vdrl = 1`) and empty VDRL test result to positive VDRL test result (`mc_cri_vdrl = 0`) if there is a link to the morbidities:
- 227
- 228
- 229
- 231
- 232
- 233
- 234
- 235
- 236
- 237
- 238
- 239
- 240
- 241
- 242
- 243
- 244
- 245
- 246
- 247
- 248
- 249
- 250
- 251
- 279

In [ ]:
tipo_morbidade_ids = [227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 279]
df_tipo_morbidade = df_tipo_morbidade[df_tipo_morbidade['mc_tmo_id'].isin(tipo_morbidade_ids)]['mc_cri_id']
df_tipo_morbidade = remove_duplicated_rows(df_tipo_morbidade, 'df_tipo_morbidade')

df_crianca.loc[df_crianca['mc_pes_id'].isin(df_tipo_morbidade.to_list()), 'mc_cri_vdrl'] = 0

vdrl_count(df_crianca, 'df_crianca')

Performing the INNER JOIN operation:

In [ ]:
from lib.dataframe_helper import inner_join

df_join_crianca_parto = inner_join(df_crianca, df_parto, 'mc_par_id')
df_join_crianca_parto_gestacao = inner_join(df_join_crianca_parto, df_gestacao, 'mc_get_id')
df_join_crianca_parto_gestacao_mulher = inner_join(df_join_crianca_parto_gestacao, df_mulher, multi_on_col=('mc_mul_id', 'mc_pes_id'))
df_join_all = inner_join(df_join_crianca_parto_gestacao_mulher, df_dado_evolutivo,  multi_on_col=('mc_mul_dado_evolutivo_atual_id', 'mc_dae_id'))

print(f'Unified dataset shape: {df_join_all.shape}')

vdrl_count(df_join_all)

Saving unified dataset into a CSV file:

In [ ]:
from lib.env_var_keys import EnvVarKeys

unified_dataset_path = os.getenv(EnvVarKeys.UNIFIED_DATASET_PATH_KEY.value)
df_join_all.to_csv(unified_dataset_path, sep=',', index=False)

print(f'Unified dataset saved to: {unified_dataset_path}')

In [ ]:
vdrl_count(df_join_all, 'df_join_all')